Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [ ]:
# PROBLEM 1
max_pool=True

In [ ]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, max_pool=False):
    if max_pool:
        # MAX_POOL.
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    else:
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
    if max_pool:
        # MAX_POOL.
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    else:
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, max_pool=max_pool)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [6]:
# Enlarge our 28x28 images to 32x32 which LeNet requires, put zeros on the pads.
train_dataset_le_net = np.pad(train_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
valid_dataset_le_net = np.pad(valid_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
test_dataset_le_net = np.pad(test_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
print('train dataset shape: %s' % str(train_dataset_le_net.shape))
print('valid dataset shape: %s' % str(valid_dataset_le_net.shape))
print('test dataset shape: %s' % str(test_dataset_le_net.shape))

train dataset shape: (200000, 32, 32, 1)
valid dataset shape: (10000, 32, 32, 1)
test dataset shape: (10000, 32, 32, 1)


In [41]:
batch_size = 16
le_net_input_size = 32

graph = tf.Graph()
with graph.as_default():
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, le_net_input_size, le_net_input_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset_le_net)
  tf_test_dataset = tf.constant(test_dataset_le_net)
  
  # Variables.
  C1_patch_size = 5
  C1_out = 6
  C1_weights = tf.Variable(tf.truncated_normal(
      [C1_patch_size, C1_patch_size, num_channels, C1_out], stddev=0.1))
  C1_biases = tf.Variable(tf.zeros([C1_out]))
  
  C2_patch_size = 5
  C2_out = 16 
  C2_weights = tf.Variable(tf.truncated_normal(
      [C2_patch_size, C2_patch_size, C1_out, C2_out], stddev=0.1))
  C2_biases = tf.Variable(tf.constant(1.0, shape=[C2_out]))
  fully_1_in=400
  fully_1_out=120
  fully_1_weights = tf.Variable(tf.truncated_normal(
      [fully_1_in, fully_1_out], stddev=0.1))
  fully_1_biases = tf.Variable(tf.constant(1.0, shape=[fully_1_out])) 
  fully_2_in=120
  fully_2_out=84
  fully_2_weights = tf.Variable(tf.truncated_normal(
      [fully_2_in, fully_2_out], stddev=0.1))
  fully_2_biases = tf.Variable(tf.constant(1.0, shape=[fully_2_out]))
  fully_3_in=84
  fully_3_weights = tf.Variable(tf.truncated_normal(
      [fully_3_in, num_labels], stddev=0.1))
  fully_3_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def my_model(data):
      C1 = tf.nn.conv2d(data, C1_weights, [1, 1, 1, 1], padding='VALID')
      pool1 = tf.nn.max_pool(tf.nn.relu(C1 + C1_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
      C2 = tf.nn.conv2d(pool1, C2_weights, [1, 1, 1, 1], padding='VALID')
      pool2 = tf.nn.max_pool(tf.nn.relu(C2 + C2_biases), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
      
      shape = pool2.get_shape().as_list()
      reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
      print("reshaped from %s to %s" % (shape, reshape.get_shape().as_list()))

      fully_1 = tf.nn.relu(tf.matmul(reshape, fully_1_weights) + fully_1_biases)
      fully_2 = tf.nn.relu(tf.matmul(fully_1, fully_2_weights) + fully_2_biases)
      return tf.matmul(fully_2, fully_3_weights) + fully_3_biases
      
  
  logits = my_model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.95, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(my_model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(my_model(tf_test_dataset))

reshaped from [16, 5, 5, 16] to [16, 400]
reshaped from [10000, 5, 5, 16] to [10000, 400]
reshaped from [10000, 5, 5, 16] to [10000, 400]


In [42]:
epochs = 1

num_steps = train_labels.shape[0] // batch_size
print("Number of epochs: %d" % epochs)
print("Number of steps in each epochs: %d" % num_steps)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    print('-------------------------------------------------------------------------------------------')
    for epoch in range(1, epochs+1):
        print('Training epoch %d...' % epoch)
        for step in range(train_labels.shape[0] // batch_size):
            offset = step * batch_size
            batch_data = train_dataset_le_net[offset:(offset + batch_size), :, :, :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if step % 1000 == 0:
                print('Minibatch loss at step %d: %f' % (step, l))
                print('Minibatch accuracy at step %d: %.1f%%' % (step, accuracy(predictions, batch_labels)))
        print('Validation accuracy at epoch %d: %.1f%%' % (epoch, accuracy(valid_prediction.eval(), valid_labels)))
        print('-------------------------------------------------------------------------------------------')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Number of epochs: 1
Number of steps in each epochs: 12500
Initialized
-------------------------------------------------------------------------------------------
Training epoch 1...
Minibatch loss at step 0: 4.614193
Minibatch accuracy at step 0: 6.2%
Minibatch loss at step 1000: 0.493333
Minibatch accuracy at step 1000: 87.5%
Minibatch loss at step 2000: 0.465030
Minibatch accuracy at step 2000: 81.2%
Minibatch loss at step 3000: 0.209469
Minibatch accuracy at step 3000: 93.8%
Minibatch loss at step 4000: 0.571026
Minibatch accuracy at step 4000: 81.2%
Minibatch loss at step 5000: 0.791286
Minibatch accuracy at step 5000: 68.8%
Minibatch loss at step 6000: 0.094960
Minibatch accuracy at step 6000: 100.0%
Minibatch loss at step 7000: 0.950561
Minibatch accuracy at step 7000: 75.0%
Minibatch loss at step 8000: 0.902424
Minibatch accuracy at step 8000: 87.5%
Minibatch loss at step 9000: 0.429893
Minibatch accuracy at step 9000: 87.5%
Minibatch loss at step 10000: 0.489706
Minibatch accur